## Real or Not? NLP with Disaster Tweets - Project 2

Big-Scale Analytics - Project 2 - Team Rolex
> Samuel Lew, Alexandre Lang, Samy Bouzerda, Alix Muller


## Data loading
We will use the iris data-set which we can directly load from sklearn library. This data-set has 3 class labels.

In [1]:
import pandas as pd
df = pd.read_csv ('../data/train.csv')
df = df[['id', 'text', 'target']]
df.iloc[:10]
index = df.index
columns = df.columns
values = df.values
df_yes = df.query('target == 1')
yes_count = 0
for row in df_yes.iterrows():
    yes_count += 1
print(f'There are {yes_count} tweets about disasters.')

def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

There are 3271 tweets about disasters.


In [5]:
df_no = df.query('target == 0')
no_count = 0
for row in df_no.iterrows():
    no_count += 1
print(f'there are {no_count} tweets that are not about disasters')

there are 4342 tweets that are not about disasters


In [4]:

import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

df = pd.read_csv('../data/train.csv', delimiter="\t")
df.head()

,"id,keyword,location,text,target"
0,"1,,,Our Deeds are the Reason of this #earthqua..."
1,"4,,,Forest fire near La Ronge Sask. Canada,1"
2,"5,,,All residents asked to 'shelter in place' ..."
3,"6,,,""13,000 people receive #wildfires evacuati..."
4,"7,,,Just got sent this photo from Ruby #Alaska..."


In [9]:
# we update and install spaCy
!pip install -U spacy

# we load the english language model
!python -m spacy download en

#load the spaCy language model
import spacy
sp = spacy.load('en_core_web_sm')

  Using cached https://files.pythonhosted.org/packages/39/c9/6c6bbb563588cd9b5d155ebda200796911588fb46a85e5f4d21b7b2ab759/spacy-2.2.4-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/6c/f9/9a5658e2f56932e41eb264941f9a2cb7f3ce41a80cb36b2af6ab78e2f8af/catalogue-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d5/7e/1981d5389b75543f950026de40a9d346e2aec7e860b2800e54e65bd46c06/blis-0.4.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/84/d1/35eab0c8cc9fd9432becaf3e90144762b3201a45079e62c47a8ae8739763/cymem-2.0.3-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/21/e1/e4e7b754e6be3a79c400eb766fb34924a6d278c43bb828f94233e0124a21/wasabi-0.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3c/5a/0d1b575ed40989d74fab25723083837c220246b25f3582917135cb32453f/preshed-3.0.2-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packag

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\importlib_metadata-0.17.dist-info\\INSTALLER'
Consider using the `--user` option or check the permissions.

C:\ProgramData\Anaconda3\python.exe: No module named spacy


ModuleNotFoundError: No module named 'spacy'

In [10]:
# Word tokenization
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

text = """When learning data science, you shouldn't get discouraged!
Challenges and setbacks aren't failures, they're just part of the journey. You've got this!"""

#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(text)

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
print(token_list)

ModuleNotFoundError: No module named 'spacy'


## KNN classifier
The k-nearest neighbors algorithm (k-NN) is a non-parametric method used for classification (and also regression). In k-NN classification, The input consists of the k closest training examples in the feature space. The output is a class membership. An object is classified by a plurality vote of its neighbors, with the object being assigned to the class most common among its k nearest neighbors.